In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('notebooks.csv')

In [5]:
#Nenhum valor nulo
df.isna().sum()

marca                0
ano                  0
qtd_nucleos          0
qtd_threads          0
ram                  0
gpu_dedicada         0
ram_gpu              0
entradas_usb         0
duracao_bateria      0
resolucao_tela       0
tipo_tela            0
tela_touch           0
armazenamento_hdd    0
armazenamento_ssd    0
segmento             0
valor                0
dtype: int64

In [6]:
# Converter valores textuas para numéricos
from sklearn.preprocessing import LabelEncoder

for col in df.columns:
    # Verifica se o tipo da coluna é object (textual)
    if df[col].dtype == 'object':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   marca              10000 non-null  int32
 1   ano                10000 non-null  int64
 2   qtd_nucleos        10000 non-null  int64
 3   qtd_threads        10000 non-null  int64
 4   ram                10000 non-null  int64
 5   gpu_dedicada       10000 non-null  int32
 6   ram_gpu            10000 non-null  int64
 7   entradas_usb       10000 non-null  int64
 8   duracao_bateria    10000 non-null  int64
 9   resolucao_tela     10000 non-null  int32
 10  tipo_tela          10000 non-null  int32
 11  tela_touch         10000 non-null  int32
 12  armazenamento_hdd  10000 non-null  int64
 13  armazenamento_ssd  10000 non-null  int64
 14  segmento           10000 non-null  int32
 15  valor              10000 non-null  int64
dtypes: int32(6), int64(10)
memory usage: 1015.8 KB


In [8]:
df.columns[-1]

'valor'

In [9]:
# Normalizar e Separar dados em Treinamento (70%), Validação (15%) e Teste (15%)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X = df[df.columns[:-1]]
y = df[[df.columns[-1]]]

scaler_X = MinMaxScaler()
X = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y = scaler_y.fit_transform(y)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# GridSearch

1. Modelo Básico:

• Crie um modelo sequencial simples com uma camada oculta e compile-o usando
MSE (Erro Quadrático Médio) como função de perda. Use o GridSearchCV para
encontrar a melhor função de ativação para essa camada oculta entre ['relu',
'tanh', 'sigmoid'].

In [10]:
from keras.models import Sequential
from keras.layers import Dense

def create_model(neurons1 = 64, activation_function = 'relu'):
    
    model = Sequential()

    model.add(Dense(neurons1, input_dim = X_train.shape[1], activation = activation_function))
    model.add(Dense(1))
    
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
    
    return model

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)

param_grid = {
    'activation_function': ['relu', 'tanh', 'sigmoid']
}

grid = GridSearchCV(estimator = model_for_grid, param_grid = param_grid, cv = 3)

grid_result = grid.fit(X_train, y_train)

print(f'Melhores parâmetros: {grid_result.best_params_}')

C:\Users\xx\AppData\Local\Temp\ipykernel_15776\113293906.py:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)


Melhores parâmetros: {'activation_function': 'relu'}


2. Otimizando a Arquitetura:

• Use o GridSearchCV para encontrar a melhor combinação de neurônios em duas
camadas ocultas. Teste combinações entre [32, 64, 128] neurônios para cada
camada.

In [13]:
def create_model(neurons1 = 64, neurons2 = 64, activation_function = 'relu'):
    
    model = Sequential()

    model.add(Dense(neurons1, input_dim = X_train.shape[1], activation = activation_function))
    model.add(Dense(neurons2, activation = activation_function))
    model.add(Dense(1))
    
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
    
    return model

In [14]:
model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)


param_grid = {
    'neurons1': [32, 64, 128],
    'neurons2': [32, 64, 128]
}

grid = GridSearchCV(estimator = model_for_grid, param_grid = param_grid, cv = 3)

grid_result = grid.fit(X_train, y_train)

print(f'Melhores parâmetros: {grid_result.best_params_}')

C:\Users\xx\AppData\Local\Temp\ipykernel_15776\319167078.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)


Melhores parâmetros: {'neurons1': 32, 'neurons2': 32}


3. Otimizando o Otimizador:

• Mantendo a arquitetura do modelo fixa, utilize o GridSearchCV para encontrar o
melhor otimizador entre ['adam', 'sgd', 'rmsprop'].

In [15]:
def create_model(neurons1 = 32, neurons2 = 32, activation_function = 'relu', optimizer = 'adam'):
    
    model = Sequential()

    model.add(Dense(neurons1, input_dim = X_train.shape[1], activation = activation_function))
    model.add(Dense(neurons2, activation = activation_function))
    model.add(Dense(1))
    
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
    
    return model

In [16]:
model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)


param_grid = {
    'optimizer': ['adam', 'sgd', 'rmsprop']
}

grid = GridSearchCV(estimator = model_for_grid, param_grid = param_grid, cv = 3)

grid_result = grid.fit(X_train, y_train)

print(f'Melhores parâmetros: {grid_result.best_params_}')

C:\Users\xx\AppData\Local\Temp\ipykernel_15776\2348818482.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)


Melhores parâmetros: {'optimizer': 'adam'}


4. Batch Size e Epochs:

• Determine o melhor batch_size e número de epochs usando o GridSearchCV.
Teste para batch_size em [32, 64, 128] e epochs em [10, 20, 30].

In [17]:
def create_model(neurons1 = 32, neurons2 = 32, activation_function = 'relu', optimizer = 'adam'):
    
    model = Sequential()

    model.add(Dense(neurons1, input_dim = X_train.shape[1], activation = activation_function))
    model.add(Dense(neurons2, activation = activation_function))
    model.add(Dense(1))
    
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
    
    return model

In [18]:
model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, batch_size = 32, verbose = 0)


param_grid = {
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 30]
}

grid = GridSearchCV(estimator = model_for_grid, param_grid = param_grid, cv = 3)

grid_result = grid.fit(X_train, y_train)

print(f'Melhores parâmetros: {grid_result.best_params_}')

C:\Users\xx\AppData\Local\Temp\ipykernel_15776\2554534723.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, batch_size = 32, verbose = 0)


Melhores parâmetros: {'batch_size': 32, 'epochs': 30}


5. Regularização:

• Use o GridSearchCV para encontrar o melhor valor para a regularização L2 (use
keras.regularizers.l2) para uma camada oculta. Teste valores como [0.01, 0.001,
0.0001].

In [19]:
from keras.regularizers import l2

def create_model(neurons1 = 32, neurons2 = 32, activation_function = 'relu', optimizer = 'adam', reg = 0.01):
    
    model = Sequential()

    model.add(Dense(neurons1, input_dim = X_train.shape[1], activation = activation_function))
    model.add(Dense(neurons2, activation = activation_function, activity_regularizer = l2(reg)))
    model.add(Dense(1))
    
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
    
    return model

In [20]:
model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, batch_size = 32, verbose = 0, reg = 0.01)


param_grid = {
    'reg': [0.01, 0.001, 0.0001]
}

grid = GridSearchCV(estimator = model_for_grid, param_grid = param_grid, cv = 3)

grid_result = grid.fit(X_train, y_train)

print(f'Melhores parâmetros: {grid_result.best_params_}')

C:\Users\xx\AppData\Local\Temp\ipykernel_15776\1185696515.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, batch_size = 32, verbose = 0, reg = 0.01)


Melhores parâmetros: {'reg': 0.01}


# RandomizedSearch

6. Função de Ativação Aleatória:

• Semelhante ao exercício 1, mas agora usando RandomizedSearchCV para
selecionar aleatoriamente a melhor função de ativação.

In [21]:
def create_model(neurons1 = 32, neurons2 = 32, activation_function = 'relu', optimizer = 'adam', reg = 0.01):
    
    model = Sequential()

    model.add(Dense(neurons1, input_dim = X_train.shape[1], activation = activation_function))
    model.add(Dense(neurons2, activation = activation_function, activity_regularizer = l2(reg)))
    model.add(Dense(1))
    
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
    
    return model

In [22]:
from sklearn.model_selection import RandomizedSearchCV

model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)

param_dist = {
    'activation_function': ['relu', 'tanh', 'softmax']
}

random_search = RandomizedSearchCV(estimator = model_for_grid, param_distributions = param_dist, n_iter = 10, cv=3)

random_result = random_search.fit(X_train, y_train)

print(f"Melhores parâmetros usando RandomizedSearchCV: {random_result.best_params_}")

C:\Users\xx\AppData\Local\Temp\ipykernel_15776\12346106.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)
c:\Users\xx\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Melhores parâmetros usando RandomizedSearchCV: {'activation_function': 'relu'}


7. Arquitetura Aleatória:

• Use o RandomizedSearchCV para encontrar uma combinação ótima de neurônios
em três camadas ocultas. Permita que qualquer valor entre 10 e 150 neurônios
seja testado para cada camada.

In [23]:
def create_model(neurons1 = 32, neurons2 = 32, neurons3 = 32, activation_function = 'relu', optimizer = 'adam', reg = 0.01):
    
    model = Sequential()

    model.add(Dense(neurons1, input_dim = X_train.shape[1], activation = activation_function))
    model.add(Dense(neurons2, activation = activation_function, activity_regularizer = l2(reg)))
    model.add(Dense(neurons3, activation = activation_function, activity_regularizer = l2(reg)))
    model.add(Dense(1))
    
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
    
    return model

In [24]:
model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)

param_dist = {
    'neurons1': [10, 50, 150],
    'neurons2': [10, 50, 150],
    'neurons3': [10, 50, 150]
}

random_search = RandomizedSearchCV(estimator = model_for_grid, param_distributions = param_dist, n_iter = 10, cv = 3)

random_result = random_search.fit(X_train, y_train)

print(f'Melhores parâmetros: {random_result.best_params_}')

C:\Users\xx\AppData\Local\Temp\ipykernel_15776\2647482244.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)


Melhores parâmetros: {'neurons3': 10, 'neurons2': 50, 'neurons1': 150}


8. Otimizador Aleatório:

• Similar ao exercício 3, mas agora usando RandomizedSearchCV.

In [25]:
def create_model(neurons1 = 150, neurons2 = 150, neurons3 = 10, activation_function = 'relu', optimizer = 'adam', reg = 0.01):
    
    model = Sequential()

    model.add(Dense(neurons1, input_dim = X_train.shape[1], activation = activation_function))
    model.add(Dense(neurons2, activation = activation_function, activity_regularizer = l2(reg)))
    model.add(Dense(neurons3, activation = activation_function, activity_regularizer = l2(reg)))
    model.add(Dense(1))
    
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
    
    return model

In [26]:
model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)

param_dist = {
    'optimizer': ['adam', 'sgd', 'rmsprop']
}

random_search = RandomizedSearchCV(estimator = model_for_grid, param_distributions = param_dist, n_iter = 10, cv = 3)

random_result = random_search.fit(X_train, y_train)

print(f'Melhores parâmetros: {random_result.best_params_}')

C:\Users\xx\AppData\Local\Temp\ipykernel_15776\1865325406.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)


Melhores parâmetros: {'optimizer': 'sgd'}


9. Iniciação de Pesos:

• Use o RandomizedSearchCV para testar diferentes inicializações de peso
['uniform', 'normal', 'zero'] em sua camada oculta.

In [27]:
def create_model(neurons1 = 10, neurons2 = 50, neurons3 = 50, activation_function = 'relu', optimizer = 'adam', reg = 0.01, kernel_initializer = 'uniform'):
    
    model = Sequential()

    model.add(Dense(neurons1, input_dim = X_train.shape[1], activation = activation_function, kernel_initializer = 'uniform'))
    model.add(Dense(1))
    
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
    
    return model

In [28]:
model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)

param_dist = {
    'kernel_initializer': ['uniform', 'normal', 'zero']
}

random_search = RandomizedSearchCV(estimator = model_for_grid, param_distributions = param_dist, n_iter = 10, cv = 3)

random_result = random_search.fit(X_train, y_train)

print(f'Melhores parâmetros: {random_result.best_params_}')

C:\Users\xx\AppData\Local\Temp\ipykernel_15776\2227252951.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0)


Melhores parâmetros: {'kernel_initializer': 'uniform'}


10. Taxa de Aprendizado:

• Se usar o otimizador 'adam', teste diferentes taxas de aprendizado entre 0.001 e 0.1
usando RandomizedSearchCV.

In [29]:
def create_model(neurons1 = 10, neurons2 = 50, neurons3 = 50, activation_function = 'relu', optimizer = 'adam', reg = 0.01, kernel_initializer = 'uniform', learning_rate = 0.001):
    
    model = Sequential()

    model.add(Dense(neurons1, input_dim = X_train.shape[1], activation = activation_function, kernel_initializer = 'uniform'))
    model.add(Dense(1))
    
    model.compile(optimizer = 'sgd', loss = 'mean_squared_error', metrics = ['accuracy'])
    
    return model

In [30]:
model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0, learning_rate = 0.001)

param_dist = {
    'learning_rate': [0.001, 0.01, 0.1]
}

random_search = RandomizedSearchCV(estimator = model_for_grid, param_distributions = param_dist, n_iter = 10, cv = 3)

random_result = random_search.fit(X_train, y_train)

print(f'Melhores parâmetros: {random_result.best_params_}')

C:\Users\xx\AppData\Local\Temp\ipykernel_15776\4103602608.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_for_grid = KerasClassifier(build_fn = create_model, epochs = 20, verbose = 0, learning_rate = 0.001)


Melhores parâmetros: {'learning_rate': 0.001}


# Otimização Bayesiana

11. Função de Ativação Bayesiana:

• Implemente uma função objetivo para otimizar a função de ativação da camada oculta e
use a Otimização Bayesiana para encontrar a melhor.

In [31]:
def objective_function(neurons1 = 32, neurons2 = 32, activation_index = 0):

    activation_functions = ['relu', 'tanh', 'sigmoid']
    activation_function = activation_functions[int(activation_index)]
    
    model = create_model(int(neurons1), int(neurons2), activation_function)
    
    history = model.fit(X_train, y_train, epochs=20, verbose=0, validation_split=0.2)
    
    val_accuracy = history.history['val_accuracy'][-1]
    
    return val_accuracy

In [36]:
from bayes_opt import BayesianOptimization

pbounds = {
    'activation_index': (0, 2)
}

optimizer = BayesianOptimization(
    f = objective_function,
    pbounds = pbounds,
    random_state = 1
)

optimizer.maximize(init_points = 5, n_iter = 10)

print(f"Melhores parâmetros usando otimização bayesiana: {optimizer.max['params']}")

|   iter    |  target   | activa... |
-------------------------------------
| 1         | 0.0007143 | 0.834     |
| 2         | 0.0007143 | 1.441     |
| 3         | 0.0007143 | 0.0002287 |
| 4         | 0.0007143 | 0.6047    |
| 5         | 0.0007143 | 0.2935    |
| 6         | 0.0007143 | 2.0       |
| 7         | 0.0007143 | 2.0       |
| 8         | 0.0007143 | 0.0002837 |
| 9         | 0.0007143 | 2.0       |
| 10        | 0.0007143 | 2.074e-05 |
| 11        | 0.0007143 | 2.0       |
| 12        | 0.0007143 | 4.594e-05 |
| 13        | 0.0007143 | 2.0       |
| 14        | 0.0007143 | 0.0003029 |
| 15        | 0.0007143 | 1.999     |
Melhores parâmetros usando otimização bayesiana: {'activation_index': 0.834044009405148}


12. Otimização da Arquitetura:

• Use a Otimização Bayesiana para encontrar a melhor combinação de neurônios em duas
camadas ocultas. Defina os limites entre 10 e 150 neurônios.

In [37]:
def objective_function(neurons1 = 32, neurons2 = 32, activation_function = 'tanh'):

    activation_function = activation_function
    
    model = create_model(int(neurons1), int(neurons2), activation_function)
    
    history = model.fit(X_train, y_train, epochs=20, verbose=0, validation_split=0.2)
    
    val_accuracy = history.history['val_accuracy'][-1]
    
    return val_accuracy

In [38]:
pbounds = {
    'neurons1': (10, 150),
    'neurons2': (10, 150),
}

optimizer = BayesianOptimization(
    f = objective_function,
    pbounds = pbounds,
    random_state = 1
)

optimizer.maximize(init_points = 5, n_iter = 10)

print(f"Melhores parâmetros usando otimização bayesiana: {optimizer.max['params']}")

|   iter    |  target   | neurons1  | neurons2  |
-------------------------------------------------
| 1         | 0.0007143 | 68.38     | 110.8     |
| 2         | 0.0007143 | 10.02     | 52.33     |
| 3         | 0.0007143 | 30.55     | 22.93     |
| 4         | 0.0007143 | 36.08     | 58.38     |
| 5         | 0.0007143 | 65.55     | 85.43     |
| 6         | 0.0007143 | 149.7     | 13.5      |
| 7         | 0.0007143 | 148.8     | 149.9     |
| 8         | 0.0007143 | 10.78     | 149.0     |
| 9         | 0.0007143 | 149.6     | 10.57     |
| 10        | 0.0007143 | 148.9     | 150.0     |
| 11        | 0.0007143 | 10.64     | 147.9     |
| 12        | 0.0007143 | 11.02     | 11.0      |
| 13        | 0.0007143 | 149.8     | 11.5      |
| 14        | 0.0007143 | 146.8     | 149.8     |
| 15        | 0.0007143 | 11.44     | 149.5     |
Melhores parâmetros usando otimização bayesiana: {'neurons1': 68.38308065836037, 'neurons2': 110.84542908190214}


13. Otimizador Bayesiano:

• Determine o melhor otimizador, transformando seus índices em um espaço contínuo e
usando Otimização Bayesiana.

In [ ]:
def objective_function(neurons1 = 68, neurons2 = 110, activation_function = 'tanh'):

    activation_function = activation_function
    
    model = create_model(int(neurons1), int(neurons2), activation_function)
    
    history = model.fit(X_train, y_train, epochs=20, verbose=0, validation_split=0.2)
    
    val_accuracy = history.history['val_accuracy'][-1]
    
    return val_accuracy

14. Dropout:

• Use a Otimização Bayesiana para encontrar a melhor taxa de dropout após cada camada
oculta. Teste valores entre 0 e 0.5.

15. Taxa de Aprendizado Bayesiana:

• Optimize a taxa de aprendizado do otimizador 'adam' usando Otimização Bayesiana. Teste
valores entre 0.0001 e 0.1